In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import random
from sklearn.metrics import precision_score, recall_score, f1_score
from web3 import Web3
import json

In [ ]:
# === Preprocess Last.fm Dataset ===
def preprocess_lastfm(path):
    df = pd.read_csv(path)
    df.rename(columns={"Username": "user_id", "Artist": "item_id"}, inplace=True)
    interaction_counts = df.groupby(['user_id', 'item_id']).size().reset_index(name='count')

    def count_to_rating(count):
        return min(5, max(1, int(np.log1p(count) / np.log(10) * 5)))

    interaction_counts['rating'] = interaction_counts['count'].apply(count_to_rating)
    interaction_counts['timestamp'] = pd.Timestamp.now().timestamp()
    return interaction_counts

# Load dataset
df = preprocess_lastfm(r"Last.fm_data.csv")

In [ ]:
# === Federated Setup ===
NUM_CLIENTS = 5
TOP_K = 5
users = df['user_id'].unique()
random.shuffle(users)
client_user_splits = np.array_split(users, NUM_CLIENTS)
client_datasets = [df[df['user_id'].isin(split)] for split in client_user_splits]


In [ ]:
# === Graph Recommender ===
def build_graph(df):
    G = nx.Graph()
    for _, group in df.groupby("user_id"):
        items = group['item_id'].tolist()
        for i in range(len(items)):
            for j in range(i + 1, len(items)):
                G.add_edge(items[i], items[j], weight=G.get_edge_data(items[i], items[j], {}).get('weight', 0) + 1)
    return G

def recommend(graph, user_df, top_k=TOP_K):
    user_items = set(user_df['item_id'])
    scores = {}
    for item in user_items:
        for neighbor in graph.neighbors(item):
            if neighbor not in user_items:
                scores[neighbor] = scores.get(neighbor, 0) + graph[item][neighbor]['weight']
    return sorted(scores, key=scores.get, reverse=True)[:top_k]

In [ ]:
# === Evaluation ===
def evaluate(client_df, train_graph, top_k=TOP_K):
    users = client_df['user_id'].unique()
    precisions, recalls, f1s, hits = [], [], [], []
    for user in users:
        user_df = client_df[client_df['user_id'] == user]
        items = user_df['item_id'].tolist()
        if len(items) < 2:
            continue
        random.shuffle(items)
        split = int(len(items) * 0.5)
        train_items = items[:split]
        test_items = items[split:]

        user_partial_df = pd.DataFrame({'user_id': [user]*len(train_items), 'item_id': train_items})
        recs = recommend(train_graph, user_partial_df, top_k)

        y_true = [1 if item in test_items else 0 for item in recs]
        y_pred = [1]*len(y_true)

        if sum(y_true) > 0:
            precisions.append(precision_score(y_true, y_pred))
            recalls.append(recall_score(y_true, y_pred))
            f1s.append(f1_score(y_true, y_pred))
            hits.append(1 if any(y_true) else 0)

    return {
        'precision': np.mean(precisions) if precisions else 0,
        'recall': np.mean(recalls) if recalls else 0,
        'f1': np.mean(f1s) if f1s else 0,
        'hit_rate': np.mean(hits) if hits else 0
    }

In [ ]:
# === Blockchain Integration ===
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
admin = w3.eth.accounts[0]
clients = w3.eth.accounts[1:6]

In [ ]:
abi = [
    {
        "inputs": [],
        "stateMutability": "nonpayable",
        "type": "constructor"
    },
    {
        "inputs": [],
        "name": "admin",
        "outputs": [
            {
                "internalType": "address",
                "name": "",
                "type": "address"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "",
                "type": "address"
            }
        ],
        "name": "balanceOf",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "name": "clients",
        "outputs": [
            {
                "internalType": "address",
                "name": "",
                "type": "address"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "decimals",
        "outputs": [
            {
                "internalType": "uint8",
                "name": "",
                "type": "uint8"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "distributeRewards",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "client",
                "type": "address"
            }
        ],
        "name": "getClientBalance",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "client",
                "type": "address"
            }
        ],
        "name": "getClientScore",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "name",
        "outputs": [
            {
                "internalType": "string",
                "name": "",
                "type": "string"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "",
                "type": "address"
            }
        ],
        "name": "performanceScores",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "client",
                "type": "address"
            }
        ],
        "name": "registerClient",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "client",
                "type": "address"
            },
            {
                "internalType": "uint256",
                "name": "f1Score",
                "type": "uint256"
            }
        ],
        "name": "submitPerformance",  # Renamed to avoid selector collision
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "symbol",
        "outputs": [
            {
                "internalType": "string",
                "name": "",
                "type": "string"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "totalSupply",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "getClients",
        "outputs": [
            {
                "internalType": "address[]",
                "name": "",
                "type": "address[]"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "client",
                "type": "address"
            }
        ],
        "name": "getPerformance",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "client",
                "type": "address"
            }
        ],
        "name": "isClientRegistered",
        "outputs": [
            {
                "internalType": "bool",
                "name": "",
                "type": "bool"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "",
                "type": "address"
            }
        ],
        "name": "performance",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "client",
                "type": "address"
            },
            {
                "internalType": "uint256",
                "name": "f1_scaled",
                "type": "uint256"
            }
        ],
        "name": "submitPerformance_v2",  # Renamed to avoid selector collision
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "",
                "type": "address"
            }
        ],
        "name": "registered",
        "outputs": [
            {
                "internalType": "bool",
                "name": "",
                "type": "bool"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]


In [ ]:
contract_address = Web3.to_checksum_address("put your address here")
contract = w3.eth.contract(address=contract_address, abi=abi)

In [ ]:
def sign_and_send(tx_func, sender):
    tx = tx_func.build_transaction({
        'from': sender,
        'nonce': w3.eth.get_transaction_count(sender),
        'gas': 3000000,
        'gasPrice': w3.to_wei('1', 'gwei')
    })
    tx_hash = w3.eth.send_transaction(tx)
    return w3.eth.wait_for_transaction_receipt(tx_hash)


In [ ]:
for client in clients:
    sign_and_send(contract.functions.registerClient(client), admin)


In [ ]:
# Build global graph and evaluate clients
global_graph = build_graph(df)
all_metrics = []
for i, client_df in enumerate(client_datasets):
    metrics = evaluate(client_df, global_graph)
    all_metrics.append(metrics)
    f1_scaled = int(metrics['f1'] * 100)
    sign_and_send(contract.functions.submitPerformance(clients[i], f1_scaled), admin)

In [ ]:
# Distribute rewards
sign_and_send(contract.functions.distributeRewards(), admin)

In [ ]:
# Final token balances
print("\n=== Final Token Balances ===")
for client in clients:
    balance = contract.functions.getClientBalance(client).call()
    print(f"{client}: {balance / 1e18:.4f} FRT")

In [ ]:
print(w3.is_connected())
print(w3.eth.get_block('latest'))


In [ ]:
print(contract.functions.getClients().call())


In [ ]:
try:
    balance = contract.functions.getClientBalance(client).call()
    print(f"{client}: {balance}")
except Exception as e:
    print(f"Error: {str(e)}")
